In [1]:
%load_ext dotenv
%dotenv

In [2]:
from datetime import datetime
import pandas as pd
from ch07_tools import get_answer
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    answer_correctness,
    context_recall,
    faithfulness,
)


from tqdm import tqdm
from utils import neo4j_driver


/Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAIBase has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAICommon has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [3]:
get_answer("How many movies are there?")

('There are 38 movies in total.',
 [{'role': 'assistant',
   'content': 'For the question: \'How many movies exist in total?\', we have the answer: \'[[{"totalMovies": 38}]]\''}])

In [4]:
test_data = pd.read_csv("../data/benchmark_data.csv", delimiter=";")
test_data.head()

,question,cypher
0,How many movies are there?,MATCH (m:Movie) RETURN count(*) AS movies
1,Who acted in the most movies?,MATCH (p:Person)-[:ACTED_IN]->(m:Movie)\nRETUR...
2,Which movies released between 1990 and 2000 ha...,MATCH (m:Movie) WHERE m.released >= 1990 AND m...
3,Who has the longest name among all actors?,MATCH (p:Person)-[:ACTED_IN]->(:Movie) RETURN ...
4,List people born before 1940.,MATCH (p:Person) WHERE p.born < 1940 RETURN p....


In [5]:
answers = []
ground_truths = []
latencies = []
contexts = []

for i, row in tqdm(test_data.iterrows(), total=len(test_data), desc="Processing rows"):
    ground_truth, _, _ = neo4j_driver.execute_query(row["cypher"])
    ground_truths.append([str(el.data()) for el in ground_truth])
    start = datetime.now()
    try:
        answer, context = get_answer(row["question"])
        context = [el['content'] for el in context]
    except Exception:
        answer, context = None, []
    latencies.append((datetime.now() - start).total_seconds())
    answers.append(answer)
    contexts.append(context)

Processing rows: 100%|██████████| 10/10 [02:54<00:00, 17.42s/it]


In [6]:
test_data['ground_truth'] = [str(el) for el in ground_truths]
test_data['answer'] = answers
test_data['latency'] = latencies
test_data['retrieved_contexts'] = contexts

In [7]:
dataset = Dataset.from_pandas(test_data.fillna("I don't know"))
result = evaluate(
    dataset,
    metrics=[
        answer_correctness,
        context_recall,
        faithfulness,
    ],
)
print(result)

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

{'answer_correctness': 0.7450, 'context_recall': 0.7500, 'faithfulness': 0.8333}


In [8]:
for key in ["answer_correctness", "context_recall", "faithfulness"]:
    test_data[key] = [el[key] for el in result.scores]
test_data

,question,cypher,ground_truth,answer,latency,retrieved_contexts,answer_correctness,context_recall,faithfulness
0,How many movies are there?,MATCH (m:Movie) RETURN count(*) AS movies,"[""{'movies': 38}""]",There are 38 movies in total.,5.073521,[For the question: 'How many movies exist in t...,0.957427,1.0,1.0
1,Who acted in the most movies?,MATCH (p:Person)-[:ACTED_IN]->(m:Movie)\nRETUR...,"[""{'p.name': 'Tom Hanks', 'movieCount': 12}""]",The actor who has acted in the most movies is ...,6.351737,[For the question: 'How many movies exist in t...,0.699149,0.5,1.0
2,Which movies released between 1990 and 2000 ha...,MATCH (m:Movie) WHERE m.released >= 1990 AND m...,"[""{'m.title': 'The Matrix', 'm.released': 1999...",The only movie released between 1990 and 2000 ...,69.407977,[For the question: 'How many movies exist in t...,0.944600,1.0,0.5
3,Who has the longest name among all actors?,MATCH (p:Person)-[:ACTED_IN]->(:Movie) RETURN ...,"[""{'name': 'Philip Seymour Hoffman'}""]",None,4.877514,[],0.178690,0.0,NaN
4,List people born before 1940.,MATCH (p:Person) WHERE p.born < 1940 RETURN p....,"[""{'p.name': 'Max von Sydow'}"", ""{'p.name': 'G...",The people born before 1940 are:\n\n- Max von ...,9.773796,[For the question: 'How many movies exist in t...,0.947563,1.0,1.0
5,Who was born in 1965 and has directed a movie?,MATCH (p:Person)-[:DIRECTED]->(m:Movie) WHERE ...,"[""{'p.name': 'Lana Wachowski'}"", ""{'p.name': '...",The people born in 1965 who have directed a mo...,23.657944,[For the question: 'How many movies exist in t...,0.693274,1.0,1.0
6,Who acted in Top Gun?,MATCH (p:Person)-[:ACTED_IN]->(m:Movie {title:...,"[""{'p.name': 'Val Kilmer'}"", ""{'p.name': 'Meg ...","The actors who acted in ""Top Gun"" are Val Kilm...",2.842534,[For the question: 'How many movies exist in t...,0.950264,1.0,1.0
7,"List the movies with a title starting with ""The""","MATCH (m:Movie) WHERE m.title STARTS WITH ""The...","[""{'m.title': 'The Birdcage'}"", ""{'m.title': '...","The movies with a title starting with ""The"" ar...",5.951908,[For the question: 'How many movies exist in t...,0.953924,1.0,1.0
8,How many movies did Tom Hanks appear in?,"MATCH (p:Person {name: ""Tom Hanks""})-[:ACTED_I...","[""{'numberOfMovies': 12}""]",None,17.414566,[],0.178237,0.0,0.0
9,Who played in the Matrix?,MATCH (p:Person)-[:ACTED_IN]->(m:Movie {title:...,"[""{'p.name': 'Emil Eifrem'}"", ""{'p.name': 'Hug...","The actors who played in ""The Matrix"" are Emil...",25.270684,[For the question: 'How many movies exist in t...,0.946806,1.0,1.0
